In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.model.eegnet import EEGNet
from pytorch_lightning.loggers import CometLogger
from src.datamodule import DataModule, NestedDataSet, DataModuleOld
from src.preprocessing import create_dataset
import pytorch_lightning as pl

In [8]:
comet_logger = CometLogger(
    api_key="WSATCNWE43zphHslQCTsJKcgk",
    workspace="marwo22", # Optional
    project_name="bachelors-project" # Optional
)

CometLogger will be initialized in online mode


In [3]:
#Load training module
# The preproccesing code is currently quite messy, still need to clean it up
train, test, val = create_dataset('./src/pickle_df')

In [4]:
# The original method, returns dataloaders which returns a 3d? tensor

dm_old = DataModuleOld(train, test, val, batch_size=16)

# For visualization purposes
x, y = None, None
for (x, y) in dm_old.train_dataloader():
    break

print(x.shape)
print(y.shape)

torch.Size([16, 64, 308])
torch.Size([16])


In [5]:
# Using my custom Dataset class, returning a 4d tensor
dm = DataModule(train, test, val, batch_size=16)

# For visualization purposes
x, y = None, None
for (x, y) in dm.train_dataloader():
    break

print(x.shape)
print(y.shape)

torch.Size([16, 1, 64, 308])
torch.Size([16])


In [6]:
model = EEGNet( chunk_size=308,
                num_electrodes=64,
                dropout=0.5,
                kernel_1=64,
                kernel_2=16,
                F1=8,
                F2=16,
                D=2,
                num_classes=2)

In [11]:
trainer = pl.Trainer(
    max_epochs=30,
    logger=comet_logger
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# Using the original dataloader method, which will error

trainer.fit(model, datamodule=dm_old)

In [12]:
# Using my custom loader, which will not error

trainer.fit(model, datamodule=dm)


  | Name   | Type       | Params
--------------------------------------
0 | block1 | Sequential | 1.6 K 
1 | block2 | Sequential | 544   
2 | lin    | Linear     | 288   
--------------------------------------
2.4 K     Trainable params
0         Non-trainable params
2.4 K     Total params
0.010     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/marwo22/bachelors-project/aef609f1348e4616a07452777482729c



Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/marwo22/bachelors-project/aef609f1348e4616a07452777482729c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_loss [21550] : (0.06267671287059784, 1.4343922138214111)
COMET INFO:     val_loss [3402]    : (0.09635721147060394, 1.1580313444137573)
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment details          : 1
COMET INFO:     filename               

In [ ]:
trainer.test(ckpt_path="best")